In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [14]:
df_Italy= pd.read_csv("df_Italy.csv" ,encoding='unicode_escape')

In [15]:
df_Italy

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Italy,2020/1/1,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Italy,2020/1/2,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Italy,2020/1/3,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Italy,2020/1/4,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Italy,2020/1/5,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2197,Italy,2021/9/24,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028064
2198,Italy,2021/9/25,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028054
2199,Italy,2021/9/26,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028045
2200,Italy,2021/9/27,60461826,69%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028044


In [4]:
#shift 21days mortality
df_Italy_mortality=df_Italy["mortality"]
df_Italy_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
          ...   
2197    0.028064
2198    0.028054
2199    0.028045
2200    0.028044
2201    0.000000
Name: mortality, Length: 2180, dtype: float64

In [13]:
#删除后21天
df_Italy.drop(df_Italy.index[2179:2201], inplace=True)
df_Italy

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Italy,2020/1/1,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Italy,2020/1/2,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Italy,2020/1/3,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Italy,2020/1/4,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Italy,2020/1/5,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174,Italy,2021/9/12,60461826,69%,58552.0,33273371.0,0.970,551.180,57793.0,0.957,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028204
2175,Italy,2021/9/13,60461826,69%,120045.0,87573881.0,1.989,1450.680,274212.0,4.542,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028195
2176,Italy,2021/9/13,60461826,69%,27498.0,33300869.0,0.456,551.636,57594.0,0.954,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028195
2177,Italy,2021/9/14,60461826,69%,318593.0,87892474.0,5.278,1455.958,274173.0,4.542,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028186


In [7]:
df_Italy1=df_Italy.join(df_Italy_mortality,how="left",rsuffix="after21days")
df_Italy1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,Italy,2020/1/1,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Italy,2020/1/2,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Italy,2020/1/3,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Italy,2020/1/4,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Italy,2020/1/5,60461826,69%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174,Italy,2021/9/12,60461826,69%,58552.0,33273371.0,0.970,551.180,57793.0,0.957,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028204,0.028204
2175,Italy,2021/9/13,60461826,69%,120045.0,87573881.0,1.989,1450.680,274212.0,4.542,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028195,0.028195
2176,Italy,2021/9/13,60461826,69%,27498.0,33300869.0,0.456,551.636,57594.0,0.954,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028195,0.028195
2177,Italy,2021/9/14,60461826,69%,318593.0,87892474.0,5.278,1455.958,274173.0,4.542,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,3.0,0.028186,0.028186


In [8]:
df_Italy1.to_csv("df_Italy_after21days",index=False)